Notebook 1: Data Preparation and Feature Engineering
Assignment: Trader Performance vs Market Sentiment

Candidate: Chakradhar Rao kallem

Objective:
- Load and clean trader data and sentiment data
- Align both datasets by date
- Create analytics-ready metrics
- Save processed outputs for analysis


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load raw datasets
trades = pd.read_csv('/content/drive/MyDrive/historical_data.csv')
sentiment = pd.read_csv('/content/drive/MyDrive/fear_greed_index.csv')

print("Trader Dataset Shape:", trades.shape)
print("Sentiment Dataset Shape:", sentiment.shape)


Trader Dataset Shape: (211224, 16)
Sentiment Dataset Shape: (2644, 4)


In [ ]:
print("TRADES DATA SAMPLE")
display(trades.head())

print("\nSENTIMENT DATA SAMPLE")
display(sentiment.head())

TRADES DATA SAMPLE


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12



SENTIMENT DATA SAMPLE


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05


In [ ]:
print("Missing Values in Trades Dataset:")
print(trades.isnull().sum())

print("\nMissing Values in Sentiment Dataset:")
print(sentiment.isnull().sum())

Missing Values in Trades Dataset:
Account             0
Coin                0
Execution Price     0
Size Tokens         0
Size USD            0
Side                0
Timestamp IST       0
Start Position      0
Direction           0
Closed PnL          0
Transaction Hash    0
Order ID            0
Crossed             0
Fee                 0
Trade ID            0
Timestamp           0
dtype: int64

Missing Values in Sentiment Dataset:
timestamp         0
value             0
classification    0
date              0
dtype: int64


In [ ]:
print("Duplicate rows in Trades:", trades.duplicated().sum())
print("Duplicate rows in Sentiment:", sentiment.duplicated().sum())

Duplicate rows in Trades: 0
Duplicate rows in Sentiment: 0


In [ ]:
trades.columns

Index(['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
       'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL',
       'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID',
       'Timestamp'],
      dtype='object')

In [ ]:
sentiment.columns

Index(['timestamp', 'value', 'classification', 'date'], dtype='object')

In [ ]:
trades[['Timestamp IST', 'Timestamp']].head()

,Timestamp IST,Timestamp
0,02-12-2024 22:50,1.730000e+12
1,02-12-2024 22:50,1.730000e+12
2,02-12-2024 22:50,1.730000e+12
3,02-12-2024 22:50,1.730000e+12
4,02-12-2024 22:50,1.730000e+12


In [ ]:
# Convert Unix timestamp (milliseconds) to datetime
trades['Timestamp'] = pd.to_datetime(trades['Timestamp'], unit='ms', errors='coerce')

# Extract date for daily alignment
trades['date'] = trades['Timestamp'].dt.date

# Convert sentiment date
sentiment['date'] = pd.to_datetime(sentiment['date'], errors='coerce').dt.date

print("Timestamp conversion completed successfully.")
print(trades[['Timestamp', 'date']].head())
print(sentiment[['date']].head())

Timestamp conversion completed successfully.
            Timestamp        date
0 2024-10-27 03:33:20  2024-10-27
1 2024-10-27 03:33:20  2024-10-27
2 2024-10-27 03:33:20  2024-10-27
3 2024-10-27 03:33:20  2024-10-27
4 2024-10-27 03:33:20  2024-10-27
         date
0  2018-02-01
1  2018-02-02
2  2018-02-03
3  2018-02-04
4  2018-02-05


In [ ]:
pd.to_datetime(trades['Timestamp'])

,Timestamp
0,2024-10-27 03:33:20
1,2024-10-27 03:33:20
2,2024-10-27 03:33:20
3,2024-10-27 03:33:20
4,2024-10-27 03:33:20
...,...
211219,2025-06-15 15:06:40
211220,2025-06-15 15:06:40
211221,2025-06-15 15:06:40
211222,2025-06-15 15:06:40


In [ ]:
print(sentiment.head())
print(sentiment['date'].min(), sentiment['date'].max())

    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03
3  1517722200     24   Extreme Fear  2018-02-04
4  1517808600     11   Extreme Fear  2018-02-05
2018-02-01 2025-05-02


In [ ]:
sentiment.tail()

,timestamp,value,classification,date
2639,1745818200,54,Neutral,2025-04-28
2640,1745904600,60,Greed,2025-04-29
2641,1745991000,56,Greed,2025-04-30
2642,1746077400,53,Neutral,2025-05-01
2643,1746163800,67,Greed,2025-05-02


In [ ]:
trades.tail()

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp,date
211219,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.101,382.2,420.80,SELL,25-04-2025 15:35,7546.6,Close Long,-20.2566,0xcd339c08dc7b615a993c0422374d8e02027400092bc2...,88803313862,False,0.042080,1.990000e+14,2025-06-15 15:06:40,2025-06-15
211220,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.101,2124.1,2338.63,SELL,25-04-2025 15:35,7164.4,Close Long,-112.5773,0x29e8ede2a3a37aa0eac00422374d8e02029b00ac9f3c...,88803313862,False,0.233863,9.260000e+14,2025-06-15 15:06:40,2025-06-15
211221,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.101,423.4,466.16,SELL,25-04-2025 15:35,5040.3,Close Long,-22.4402,0x0780085b0c0a943eea800422374d920204c100edf579...,88803313862,False,0.046616,6.930000e+14,2025-06-15 15:06:40,2025-06-15
211222,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.101,3599.8,3963.38,SELL,25-04-2025 15:35,4616.9,Close Long,-190.7894,0x349c29934913b25c89e20422374d920204cd008b8a0e...,88803313862,False,0.396337,4.180000e+14,2025-06-15 15:06:40,2025-06-15
211223,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.101,1017.1,1119.83,SELL,25-04-2025 15:35,1017.1,Close Long,-53.9063,0xac77fab973c455d77a670422374d9602039800f1f78c...,88803313862,False,0.111982,3.270000e+14,2025-06-15 15:06:40,2025-06-15


In [ ]:
# Get maximum sentiment date
max_sentiment_date = sentiment['date'].max()

# Filter trades only within sentiment range
trades = trades[trades['date'] <= max_sentiment_date]

print("Filtered Trades Shape:", trades.shape)
print("New Trade Date Range:")
print(trades['date'].min(), "to", trades['date'].max())


Filtered Trades Shape: (184263, 17)
New Trade Date Range:
2023-03-28 to 2025-02-19


In [ ]:
print("Trades Date Range:")
print(trades['date'].min(), "to", trades['date'].max())

print("\nSentiment Date Range:")
print(sentiment['date'].min(), "to", sentiment['date'].max())


Trades Date Range:
2023-03-28 to 2025-02-19

Sentiment Date Range:
2018-02-01 to 2025-05-02


In [ ]:
# Standardize trade column names
trades.columns = trades.columns.str.strip().str.lower().str.replace(" ", "_")

# Standardize sentiment column names
sentiment.columns = sentiment.columns.str.strip().str.lower().str.replace(" ", "_")

print("Trades Columns:\n", trades.columns)
print("\nSentiment Columns:\n", sentiment.columns)

Trades Columns:
 Index(['account', 'coin', 'execution_price', 'size_tokens', 'size_usd', 'side',
       'timestamp_ist', 'start_position', 'direction', 'closed_pnl',
       'transaction_hash', 'order_id', 'crossed', 'fee', 'trade_id',
       'timestamp', 'date'],
      dtype='object')

Sentiment Columns:
 Index(['timestamp', 'value', 'classification', 'date'], dtype='object')


In [ ]:
sentiment.columns

Index(['timestamp', 'value', 'classification', 'date'], dtype='object')

In [ ]:
merged = trades.merge(
    sentiment,
    left_on='date',
    right_on='date',
    how='left'
)

print("Merged Shape:", merged.shape)
print("Missing Sentiment Rows:", merged['classification'].isnull().sum())

Merged Shape: (184263, 20)
Missing Sentiment Rows: 0


In [ ]:
# Create win flag
merged['is_win'] = merged['closed_pnl'] > 0

# Create long flag
merged['is_long'] = merged['side'].str.lower().apply(lambda x: 1 if x == 'buy' else 0)

# Build trader-day level metrics
daily_metrics = merged.groupby(['account', 'date', 'classification']).agg(
    daily_pnl=('closed_pnl', 'sum'),
    trade_count=('trade_id', 'count'),
    total_volume_usd=('size_usd', 'sum'),
    avg_trade_size_usd=('size_usd', 'mean'),
    win_rate=('is_win', 'mean'),
    long_ratio=('is_long', 'mean'),
    total_fees=('fee', 'sum')
).reset_index()

print("Daily Metrics Shape:", daily_metrics.shape)
daily_metrics.head()

Daily Metrics Shape: (77, 10)


,account,date,classification,daily_pnl,trade_count,total_volume_usd,avg_trade_size_usd,win_rate,long_ratio,total_fees
0,0x083384f897ee0f19899168e3b1bec365f52a9012,2024-10-27,Greed,-3.275059e+05,462,6842632.02,14810.891818,0.025974,0.300866,1112.895650
1,0x083384f897ee0f19899168e3b1bec365f52a9012,2025-02-19,Fear,1.927736e+06,3356,54854631.95,16345.241940,0.405542,0.468415,6292.416654
2,0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,2024-10-27,Greed,2.060745e+04,320,797679.97,2492.749906,0.531250,0.453125,202.167151
3,0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,2025-02-19,Fear,1.709873e+04,3533,5982611.50,1693.351684,0.435041,0.432494,1664.914670
4,0x271b280974205ca63b716753467d5a371de622ab,2024-07-03,Neutral,-1.000000e+00,5,30351.00,6070.200000,0.000000,0.400000,10.197932


In [ ]:
print("Unique Accounts:", trades['account'].nunique())
print("Unique Dates:", trades['date'].nunique())
print("Unique Account-Date combinations:",
      trades.groupby(['account', 'date']).ngroups)

Unique Accounts: 32
Unique Dates: 6
Unique Account-Date combinations: 77


In [ ]:
print("Date Distribution:")
print(trades['date'].value_counts().head(10))

Date Distribution:
date
2025-02-19    133871
2024-10-27     35241
2024-07-03      7141
2024-03-09      6962
2023-11-14      1045
2023-03-28         3
Name: count, dtype: int64


In [ ]:
# Trader-level risk metrics
trader_risk = daily_metrics.groupby('account').agg(
    pnl_volatility=('daily_pnl', 'std'),
    avg_daily_pnl=('daily_pnl', 'mean'),
    avg_win_rate=('win_rate', 'mean'),
    avg_trade_count=('trade_count', 'mean')
).reset_index()

trader_risk.head()

,account,pnl_volatility,avg_daily_pnl,avg_win_rate,avg_trade_count
0,0x083384f897ee0f19899168e3b1bec365f52a9012,1.594697e+06,800114.909989,0.215758,1909.0
1,0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,2.481039e+03,18853.086574,0.483146,1926.5
2,0x271b280974205ca63b716753467d5a371de622ab,1.681032e+04,10587.696144,0.410153,382.0
3,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,7.880372e+04,66157.740843,0.438246,6633.0
4,0x2c229d22b100a7beb69122eed721cee9b24011dd,4.539380e+04,84313.992027,0.504117,1617.0


In [ ]:
import os

os.makedirs('csv_files', exist_ok=True)

daily_metrics.to_csv('/content/drive/MyDrive/csv_files/daily_trader_metrics.csv', index=False)
trader_risk.to_csv('/content/drive/MyDrive/csv_files/trader_risk_profile.csv', index=False)

print("Processed files saved.")

Processed files saved.
